# Programación declarativa @ URJC
# Programación funcional
## Curso 21-22, convocatoria ordinaria (27 de octubre de 2021)
## Campus de Vicálvaro


# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [156]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined object Tree
import Tree._


defined function foldTree

In [157]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
    
    abstract class Option[A]{
        // Test whether the value is defined (i.e. `Some`) or not (i.e. `None`)
        def isDefined: Boolean 
        def map[B](f: A => B): Option[B]
    }
    
    abstract class Either[A, B]{
        // Test whether the value is left or right
        def isLeft: Boolean 
        def isRight: Boolean 
        def map[C](f: B => C): Either[A, C]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [158]:
type Not[P] = P => Nothing


defined type Not

# Ejercicio 1 (variante 1)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar la siguiente tautología de la lógica proposicional intuicionista: 

$\neg p \rightarrow \neg\neg\neg p$

In [65]:
def proof[P]: Not[P] => Not[Not[Not[P]]] = 
    (notP: P => Nothing) => 
        (f: (P => Nothing) => Nothing) => 
            f(notP) : Nothing

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$(\neg q \rightarrow \neg p) \rightarrow (p \rightarrow q)$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $q$, es decir, que la fórmula $q \vee \neg q$  puede utilizarse como premisa.


In [62]:
def proof[P, Q](middle: Either[Q, Not[Q]]): (Not[Q] => Not[P]) => (P => Q) = 
    (f: Not[Q] => Not[P]) => (p: P) => 
        middle match {
            case Left(q: Q) => 
                q : Q
            case Right(notQ: Not[Q]) => 
                f(notQ)(p) : Q
        }

defined function proof

# Ejercicio 2
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(1+1)^X \cong Boolean^X$

de tal forma que se verifique el siguiente test unitario para $X=Int$:

_(variante 1)_

In [171]:
class IsoTest(
    from: Either[Int, Either[Unit, Unit] => Int] => (Int, Option[Int]), 
    to: ((Int, Option[Int])) => Either[Int, Either[Unit, Unit] => Int]
) extends FlatSpec with Matchers{
    
    "from-to" should "work" in {
        from(to((0, None))) shouldBe (0, None)
        from(to((0, Some(1)))) shouldBe (0, Some(1))
    }
    
    val f: Either[Unit, Unit] => Int = {
        case Left(()) => 0
        case Right(()) => 1
    }
    
    "to-from" should "work" in {
        to(from(Left(0))) shouldBe Left(0)
        to(from(Right(f))).map(_(Left(()))) shouldBe Right(f).map(_(Left(())))
        to(from(Right(f))).map(_(Right(()))) shouldBe Right(f).map(_(Right(())))
    }
}

defined class IsoTest

In [172]:
def from[X](l: Either[X, Either[Unit, Unit] => X]): (X, Option[X]) =
    l match {
        case Left(x) => (x, None)
        case Right(f) => (f(Left(())), Some(f(Right(()))))
    }

defined function from

In [173]:
def to[X](l: (X, Option[X])): Either[X, Either[Unit, Unit] => X] =
    l match {
        case (x, None) => Left(x)
        case (x1, Some(x2)) => Right{
            case Left(()) => x1
            case Right(()) => x2
        }
    }

defined function to

In [174]:
run(new IsoTest(from[Int], to[Int]))

cmd170$Helper$IsoTest:
from-to
- should work
to-from
- should work


# Ejercicio 3
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores de tipo `X` o `Y` y devuelve una lista con los valores de tipo `Y` pertenecientes a la lista de entrada (en el mismo orden), _en caso de que dicha lista no contenga ningún valor de tipo `X`_; en caso de que sí lo contenga, la función `sequence` devolverá el primer valor de tipo `X` encontrado. El comportamiento de la función se ilustra en el siguiente test unitario, donde la función `sequence` se encuentra particularizada para los tipos `X=String` e `Y=Int`:


In [137]:
class TestSlice(
    slice: List[Int] => (Int, Int) => List[Int]
) extends FlatSpec with Matchers{
    "slice" should "work" in {
        slice(List())(0,3) shouldBe List()
        slice(List(1,2,3,4))(5,6) shouldBe List()
        slice(List(1,2,3,4))(0,2) shouldBe List(1,2,3)
        slice(List(1,2,3,4))(0,6) shouldBe List(1,2,3,4)
        slice(List(1,2,3,4))(1,3) shouldBe List(2,3,4)
        slice(List(1,2,3,4))(1,2) shouldBe List(2,3)
    }
}

defined class TestSlice

__a) (1 punto)__ Implementa la función `slice` con recursión final.

In [176]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = {
    def auxSlice(aux: List[A])(out: List[A], idx: Int): List[A] = 
        aux match {
            case e :: tail if from <= idx && idx <= to => 
                auxSlice(tail)(e :: out, idx + 1)
            case _ :: tail if idx < from => 
                auxSlice(tail)(out, idx + 1)
            case _ => 
                out
        }
    
    auxSlice(list)(Nil, 0).reverse
}

defined function slice

In [177]:
run(new TestSlice(slice))

cmd136$Helper$TestSlice:
slice
- should work


__b) (1 punto)__ Implementa la función `slice` con `foldLeft`.

In [178]:
def slice[A](list: List[A])(from: Int, to: Int): List[A] = 
    list.foldLeft((Nil: List[A], 0)){
        case ((out, idx), e) if from <= idx && idx <= to => 
            (e :: out, idx + 1)
        case ((out, idx), _) => 
            (out, idx + 1)
    }._1.reverse

defined function slice

In [179]:
run(new TestSlice(slice))

cmd136$Helper$TestSlice:
slice
- should work


# Ejercicio 4
__(3 puntos)__

Hallar el camino más largo entre la raíz y las hojas de un árbol binario. 

In [180]:
class TestLongestPath(longest: Tree[Int] => List[Int]) extends FlatSpec with Matchers{
    "longest path" should "work" in {
        longest(void) shouldBe 
            List()
        longest(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe 
            List(5,4,3,2,1)
        longest(node(left(leaf(4), 1), 0, 
                     node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe 
            List(0, 2, 2, 4, 5)
    }
}

defined class TestLongestPath

__a) (1,5 puntos)__ Recursivamente.

In [181]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    tree match {
        case Empty() => List()
        case Node(left, root, right) => 
            val longestLeft: List[A] = longestPath(left)
            val longestRight: List[A] = longestPath(right)
            root :: (if (longestLeft.length > longestRight.length) longestLeft 
                     else longestRight)
    }

defined function longestPath

In [43]:
run(new TestLongestPath(longestPath))

cmd40$Helper$TestLongestPath:
longest path
- should work


__b) (1,5 puntos)__ Con foldTree.

In [182]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)(List[A]())(
        (longestLeft, root, longestRight) => 
            root :: (if (longestLeft.length > longestRight.length) longestLeft 
                     else longestRight)
    )

defined function longestPath

Sin utilizar la función `length`

In [51]:
def longestPath[A, B](tree: Tree[A]): List[A] = 
    foldTree(tree)((0, List[A]())){
        case ((l, longestLeft), root, (r, longestRight)) => 
            if (l > r) (l+1, root :: longestLeft) 
            else (r+1, root :: longestRight)
    }._2

defined function longestPath

In [50]:
run(new TestLongestPath(longestPath))

cmd40$Helper$TestLongestPath:
longest path
- should work
